In [1]:
class GaussInt:
    def __init__(self, real, imaginary=None):
        # allow native complex type construction or parameter construction
        if (imaginary == None):
            self.r = int(real.real)
            self.i = int(real.imag)
        else:
            self.r = real
            self.i = imaginary
        
    def __mul__(self, other):
        # construct GaussInt from non-GaussInt
        if not isinstance(other, GaussInt):
            other = GaussInt(other)      

        # (a + bi) (c + di) = ac - bd
        a, b, c, d = self.r, self.i, other.r, other.i
        r = (a * c) - (b * d)
        i = (a * d) + (b * c)
        return GaussInt(r, i)
    
    def divide(self, other):
        if not isinstance(other, GaussInt):
            r_r = int(round(self.r / other))
            r_i = int(round(self.i / other))
            return GaussInt(r_r, r_i), self - (GaussInt(r_r, r_i) * other)
        else:
            a, b, c, d = self.r, self.i, other.r, other.i
            conj = GaussInt(other.r, -other.i)
            # (a + bi) / (c + di)
            # (a + bi) * (c - di) / ((c + di) * (c - di))
            # (a + bi) * (c + di) / (c ** 2 - d ** 2)
            res = (self * conj) / (other * conj).r
            
            r_r = int(round(res.r))
            r_i = int(round(res.i))
            return GaussInt(r_r, r_i), self - (GaussInt(r_r, r_i) * other)
    
    def __truediv__(self, other):
        return self.divide(other)[0]
    
    def __mod__(self, other):
        return self.divide(other)[1]
        
    def __sub__(self, other):
        return self + (other * -1)
    
    def __add__(self, other):
        if not isinstance(other, GaussInt):
            other = GaussInt(other)
            
        return GaussInt(self.r + other.r, self.i + other.i)

    def __repr__(self):
        return "({}{}{}i)".format(self.r, "+" if (self.i >= 0) else "", self.i)
    
    def __eq__(self, other):
        # construct GaussInt from non-GaussInt
        if not isinstance(other, GaussInt):
            other = GaussInt(other)
        return (self.r == other.r) and (self.i == other.i)
    
    def __neq__(self, other):
        return not (self == other)
    
    def norm(self):
        return self.r ** 2 + self.i ** 2

a = GaussInt(5+6j)
b = GaussInt(1+2j)

(a % 10)

(5-4i)